# 1. Variables and Aliases

## Shell (Local) Variables

Scope is in the **current** shell session (interactive shell session or a script).

## Environment (Global) Variables

Variable which has been exported. This means that it will be visible as a variable, not only in the shell session 
that created it, but also for any process (not just shells) that are started from that session.

You can use the `env` command to see all created environmental variables.

**Resources**

[Stackpost explaing this](https://unix.stackexchange.com/questions/363525/what-is-the-difference-in-usage-between-shell-variables-and-environment-variable)

### Example


`test1.sh`
```bash

export var1=pizza
var2=cheese

test2.sh
```

`test1.sh`
```bash
echo ${var1} # will work
echo ${var2} # will not find
```

## The `$PATH` Variable

This variable controls how your computer searches for programs/software.

```bash

# See all paths in our $PATH var
echo ${PATH} | sed -e "s/:/\n/g"
```

Let's say that we download a new package to use the tool `GATK`. The tool is installed in `/home/Desktop/gatk/bin/gatk.sh`

If we want our computer to know to run `gatk` without typing in the full path, we have to add it to our `$PATH` variable.

Use `=` to append new parts of the path to the existing one using a colon, `:`. Note: there can be no spaces on either side of the `=`.

```bash
PATH=$PATH:/home/Desktop/gatk/bin/
```

## Aliases

Exactly what they sound like, they allow us to set a shortcut command for something

Let's say I wan't an easier command to ssh into OHSU's exahead server using a proxyjump through the acc with X11 forwarding.

```bash
alias exahead="ssh -X exahead1 -J ${USERNAME}@acc.ohsu.edu"

exahead
```

## Functions

Sort or of similar to aliases, but are incredibly flexible and can do a lot

```bash
exahead() {
    user=$1
    
    if [ ${user} == ${USERNAME} ]; then
        echo "Setting default username"
    fi
    
    ssh -X exahead1 ${user}@acc.ohsu.edu
}
```

If you find yourself typing the same long(ish) command over and over... 
    
### <span style="color:cyan">**WRITE A FUNCTION FOR IT**</span>

But `where` and `how` do we save these functions and aliases so that they are resuable?

# 1. The Bash Profile

## *.bash_profile*

- The `.` means it is a hidden file (not listed using `ls` unless we do `ls -a`)
    - Also sometimes called a `dot file`
- This file **runs at the start of a new login shell**
- Common things to have in here include adding items to the our `$PATH` variable
    - We don't need to append our path every time we create a new instance of bash

## *.bashrc* (*.zshrc*)

- This file **runs every time we start an interactive shell**
- This is a great place to put custom functions and aliases

[More detail in this stackexchange post](https://unix.stackexchange.com/questions/129143/what-is-the-purpose-of-bashrc-and-how-does-it-work)

#### Making and loading changes to the `.bashrc`

To reload changes type,

```bash
source ~/.bashrc
```

## Setting `PYTHONPATH`

There are a couple of things we have to do in windows to find and set our python interpreter.

Add the following lines into your `.bashrc`

```bash

PYTHONPATH=/c/Users/[USER]/anaconda3/python.exe
alias python="winpty ${PYTHONPATH}"

```

Source reads and runs commands and variable definitions from filename in the `current shell environment`

One of the reasons we might do this is to load variables into our current shell environment

# Reproducibility Systems & Remote Dev

### My Setup

<center><img src="./images/my_dev_setup.png"/ width="750"></center>


## Screen based text editors 

<center><img src="https://upload.wikimedia.org/wikipedia/commons/9/9f/Vimlogo.svg"/ width="250"></center>
 
 #### Why use them?
 
 - They are installed *by default* on most UNIX systems
 - Sometimes they are the only option (without X11 forwarding) 
     - Advanced Computing Center (exahead)
     - AWS
 - Extremely lightweight 
 - Highly customizable
 - **EXTREMELY FAST**
     
Alternatives to vim

- `vim` is an extension of `vi` and `neovim` an extension `vim`

- Emacs, nano

### Basic concepts of vim

#### Modes

**FIRST**

To exit vim we will enter `normal mode` (hit `<Esc>`) and type,
- `:q!` (quit without saving)  
- `:x!` (save and quit)
- `:w!` (write and do not quit)
 

In vim we enter `modes` there are 3 modes we will briefly mention

`normal mode`
- In normal mode we are not editing text, we are moving around or can delete text
- We access normal mode by hitting the `<Esc>` key

#### Movement in vim

<center><img src="https://miro.medium.com/max/588/1*SIeiGIt9VvMPXfoUC2yQrw.png"/ width="250"></center>

| command | description                                  |
|---------|----------------------------------------------|
| b       | go to beginning of word                      |
| w       | go to end of word                            |
| t[char] | move to next instance of [char]              |
| T[char] | move to previous instance of [char]          |
| o       | insert newline below and enter `insert mode` |
| O       | insert newline above and enter `insert mode` |
| 4j      | move down 4 lines                            |


`visual mode` (from normal mode hit `v`)
- Used to "highlight" text like we would by clicking and dragging

`insert mode` (from normal mode hit `i`)
- Make changes 

## Python (conda environements)

Conda environments allow us to install instances of python (and specified libraries) to promote reproducibility and ensure consistent development across different users or systems.

**my_conda_env.yaml**
```python

name: bash_env
channels:
  - defaults
dependencies:
  - python=3.8.5
  - scipy=1.5.2
  - tqdm=4.50.2
  - pandas=1.1.3
  - numpy=1.19.2
  - jupyter
  - pip=20.2.4

```

Create the env by running,

```bash

# Create the environment
conda env create -f my_conda_env.yaml

# Activate it
conda activate bash_env

# Update 
conda env update -f my_conda_env.yaml

# Remove a library
conda remove pandas

# Remove entire environment
conda env remove -n bash_env

# List installed environments
conda list --envs

```

## Multiplexers (tmux)

Multiplexers (like tmux) allow you to tile windows in a command-line environment. Allowing you to run multiple programs within one terminal.

Tmux is great for,

- Creating a "pseudo"-ide like dev environment
    - Editing scripts, running them, and exploring variables
- Moving in and out of multiple instance of different bash-like operations
- Working remotely (via ssh) and locally in a single terminal
- Leaving instances of bash running on HPC clusters (like exahead)

#### Primary tmux commands

```bash

# CREATE a new session of tmux
tmux new-session -s [session_name]

# ATTACH an existing tmux session
tmux attach-session -t [session_name]

# KILL a running tmux session
tmux kill-session -t [session_name]

```

The other key component of tmux is how you interact with a tmux session, by default `<ctrl+b>` will let your computer now the next keypress is associated with a tmux command.

`<ctrl+b>` then press `d` will `detach` the current tmux session and bring you back to where you were before
`<ctrl+b>` then press `->` (arrow key) will move you to the next pane in your tmux session (if you have multiple)

There a ton of tmux commands so I bookmark this [cheatsheet](https://tmuxcheatsheet.com/)


# Installing tmux

### Windows

**Using git-bash terminal**

```bash
# Download the tool unzip in ZStandard compression format
curl -L https://sourceforge.net/projects/zstd-for-windows/files/zstd_Windows7%5BMinGW%5D%28static%29.zip -o zstd.zip

# Unzip the tool
unzip zstd.zip

# Move it to our readily accessible path
mv zstd_Win32/zstd.exe /c/[User]/Windows/

# Double check its available
which zstd

# Download tmux, libevent, zsh from the MYS2 dir
curl -L https://repo.msys2.org/msys/x86_64/tmux-3.3.a-1-x86_64.pkg.tar.zst -o tmux.tar.zst
curl -L https://repo.msys2.org/msys/x86_64/libevent-2.1.12-2-x86_64.pkg.tar.zst -o libevent.tar.zst

# Extract from Zstandard
zstd -d tmux.tar.zst
zstd -d libevent.tar.zst

# Extract from tar
tar -xvf tmux.tar.zst -C "/c/Program Files/Git/"
tar -xvf libevent.tar.zst -C "/c/Program Files/Git/"

```

### MAC

```bash

# If you dont have homebrew
/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

# Install brew 
brew update

# Install tmux through brew
brew install tmux

```


# ANSWER

`prep_data.sh`
```bash
#!/bin/bash

# Download the file if it doesn't exist 
    # Print if it does exist
if [[ ! -e statelist.txt ]]; then
    echo "Downloading file..."
    curl "https://gist.githubusercontent.com/bensie/130828/raw/43d59239b4c1c40672dc6eea0f4565a8915b5372/states.txt" -o statelist.txt
else
    echo "File already exists..."
fi

logfile="k_states.txt"

if [[ -e ${logfile} ]]; then
    echo "Removing logfile..."
    rm ${logfile}
fi
```

`k_checker.sh`
```bash
#!/bin/bash

IFS=$'\n'

# Write the lines of ONLY THE US states that contain a K to a file called k_states.txt
for ii in `cat statelist.txt`; do

    # Check whether upper or lowercase k exists
    if [[ ${ii} = *k* ]] || [[ ${ii} = *k* ]]; then
    
        # Make sure Canada isnt in there
        if [[ ${ii} != *Canada* ]]; then
        
            # Change '' to "" makes it easier (other solutions)
            ii=`echo ${ii} | sed "s/'/\"/g"` 
            
            # Get 4th item of "
            k_state=`echo "$ii" | awk -F'"' '{print $4}'` 
            
            # Print to our log
            echo ${k_state} >> k_states.txt
        fi
    fi
done
```

`run.sh`
```bash

#!/bin/bash

par_dir="pathtofolder"

${par_dir}/prep_data.sh

${par_dir}/k_checker.sh

if [[ -e k_states.txt ]]; then
    echo `wc -l k_states.txt`
fi

```
